In [21]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.chains import RetrievalQA

In [22]:
loader = PyMuPDFLoader("Virtual_characters.pdf")
PDF_data = loader.load()

In [23]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=5)
all_splits = text_splitter.split_documents(PDF_data)

In [24]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cpu'}
embedding = HuggingFaceEmbeddings(model_name=model_name,
                                  model_kwargs=model_kwargs)

vectordb = Chroma.from_documents(documents=all_splits, embedding=embedding, persist_directory=persist_directory)

Exception ignored in: <function Llama.__del__ at 0x000001B069CC36D0>
Traceback (most recent call last):
  File "c:\Users\April\anaconda3\envs\visual-inspetion\lib\site-packages\llama_cpp\llama.py", line 1972, in __del__
    self.close()
  File "c:\Users\April\anaconda3\envs\visual-inspetion\lib\site-packages\llama_cpp\llama.py", line 1969, in close
    self._stack.close()
AttributeError: 'Llama' object has no attribute '_stack'
Exception ignored in: <function Llama.__del__ at 0x000001B069CC36D0>
Traceback (most recent call last):
  File "c:\Users\April\anaconda3\envs\visual-inspetion\lib\site-packages\llama_cpp\llama.py", line 1972, in __del__
    self.close()
  File "c:\Users\April\anaconda3\envs\visual-inspetion\lib\site-packages\llama_cpp\llama.py", line 1969, in close
    self._stack.close()
AttributeError: 'Llama' object has no attribute '_stack'


In [35]:
import os

# model_path = "llama.cpp/models/llama-2-7b-chat/llama-2_q4.gguf"
model_path = "C:/Users/April/llama.cpp/models/Taiwan-LLM-7B/Breeze-7B-Instruct-v0.1-Q4_K_M.gguf"
if not os.path.exists(model_path):
    print("Model path does not exist:", model_path)
else:
    print("Model path is correct")

Model path is correct


In [36]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import LlamaCpp
model_path = "C:/Users/April/llama.cpp/models/Taiwan-LLM-7B/Breeze-7B-Instruct-v0.1-Q4_K_M.gguf"

llm = LlamaCpp(
    model_path = model_path,
    n_gpu_layers=100,
    n_batch=512,
    n_ctx=2048,
    f16_kv=True,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
    verbose=True,
)

llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from C:/Users/April/llama.cpp/models/Taiwan-LLM-7B/Breeze-7B-Instruct-v0.1-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = ..
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:     

In [37]:
from langchain.chains import LLMChain
from langchain.chains.prompt_selector import ConditionalPromptSelector
from langchain.prompts import PromptTemplate

DEFAULT_LLAMA_SEARCH_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""<<SYS>> 
    You are a helpful assistant eager to assist with providing better Google search results.
    <</SYS>> 
    
    [INST] Provide an answer to the following question in 150 words. Ensure that the answer is informative, \
            relevant, and concise:
            {question} 
    [/INST]""",
)

DEFAULT_SEARCH_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are a helpful assistant eager to assist with providing better Google search results. \
        Provide an answer to the following question in about 150 words. Ensure that the answer is informative, \
        relevant, and concise: \
        {question}""",
)

QUESTION_PROMPT_SELECTOR = ConditionalPromptSelector(
    default_prompt=DEFAULT_SEARCH_PROMPT,
    conditionals=[(lambda llm: isinstance(llm, LlamaCpp), DEFAULT_LLAMA_SEARCH_PROMPT)],
)

prompt = QUESTION_PROMPT_SELECTOR.get_prompt(llm)
prompt

PromptTemplate(input_variables=['question'], template='<<SYS>> \n    You are a helpful assistant eager to assist with providing better Google search results.\n    <</SYS>> \n    \n    [INST] Provide an answer to the following question in 150 words. Ensure that the answer is informative,             relevant, and concise:\n            {question} \n    [/INST]')

In [38]:
llm_chain = LLMChain(prompt=prompt, llm=llm)
question = "What is Taiwan known for?"
llm_chain.invoke({"question": question})

c:\Users\April\anaconda3\envs\visual-inspetion\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
Exception ignored in: <function Llama.__del__ at 0x000001B069CC36D0>
Traceback (most recent call last):
  File "c:\Users\April\anaconda3\envs\visual-inspetion\lib\site-packages\llama_cpp\llama.py", line 1972, in __del__
    self.close()
  File "c:\Users\April\anaconda3\envs\visual-inspetion\lib\site-packages\llama_cpp\llama.py", line 1969, in close
    self._stack.close()
AttributeError: 'Llama' object has no attribute '_stack'


 Taiwan is known for its rich cultural heritage, breathtaking natural beauty, diverse cuisine, advanced technology, and thriving economy. The island nation boasts a unique blend of ancient traditions and modern advancements, making it an exciting destination for travelers and businesses alike.


llama_print_timings:        load time =    7476.07 ms
llama_print_timings:      sample time =      15.87 ms /    56 runs   (    0.28 ms per token,  3529.34 tokens per second)
llama_print_timings: prompt eval time =    7475.66 ms /    81 tokens (   92.29 ms per token,    10.84 tokens per second)
llama_print_timings:        eval time =   10429.69 ms /    55 runs   (  189.63 ms per token,     5.27 tokens per second)
llama_print_timings:       total time =   18177.01 ms /   136 tokens


{'question': 'What is Taiwan known for?',
 'text': ' Taiwan is known for its rich cultural heritage, breathtaking natural beauty, diverse cuisine, advanced technology, and thriving economy. The island nation boasts a unique blend of ancient traditions and modern advancements, making it an exciting destination for travelers and businesses alike.'}

In [39]:
retriever = vectordb.as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [40]:
query = "Tell me about Alison Hawk's career and age"
qa.invoke(query)



> Entering new RetrievalQA chain...


Llama.generate: prefix-match hit


 Alison Hawk is a 28-year-old female researcher whose genius and mysterious intentions make her stand out in her career.


llama_print_timings:        load time =    7476.07 ms
llama_print_timings:      sample time =       7.32 ms /    30 runs   (    0.24 ms per token,  4096.12 tokens per second)
llama_print_timings: prompt eval time =    6378.99 ms /   157 tokens (   40.63 ms per token,    24.61 tokens per second)
llama_print_timings:        eval time =    4577.44 ms /    29 runs   (  157.84 ms per token,     6.34 tokens per second)
llama_print_timings:       total time =   11010.29 ms /   186 tokens



> Finished chain.


{'query': "Tell me about Alison Hawk's career and age",
 'result': ' Alison Hawk is a 28-year-old female researcher whose genius and mysterious intentions make her stand out in her career.'}